In [29]:
import pandas as pd
import numpy as np
import time
import datetime

In [4]:
kaoqin=pd.read_excel('./20201016课前资料/data/考勤201909.xlsx',sheet_name='201909')

In [7]:
kaoqin=kaoqin.iloc[:,1:]

In [14]:
kaoqin['day']=kaoqin['DATETIME'].dt.date

In [20]:
#设置上班时间和下班时间
sign_in=kaoqin.groupby(['day','ID','NAME'])['DATETIME'].min()
sign_out=kaoqin.groupby(['day','ID','NAME'])['DATETIME'].max()

In [34]:
#拼为dataframe
df=pd.DataFrame({
    'sign_in':sign_in,
    'sign_out':sign_out
})

In [59]:
datestringlist=[str(x)+' 15:00:00' for x in df['sign_in'].dt.date.values]
tickle_in=[datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S') for x in datestringlist]

In [61]:
df['sign_in_tickle']=tickle_in

In [62]:
df

sign_in            sign_out      sign_in_tickle
day        ID NAME                                                            
2019-09-01 2  王明浩  2019-09-01 08:47:00 2019-09-01 17:31:00 2019-09-01 15:00:00
           6  赵佳乐  2019-09-01 08:46:00 2019-09-01 20:10:00 2019-09-01 15:00:00
           15 李颖   2019-09-01 08:29:00 2019-09-01 17:35:00 2019-09-01 15:00:00
           16 孙超   2019-09-01 08:39:00 2019-09-01 17:31:00 2019-09-01 15:00:00
           19 刘小虹  2019-09-01 08:33:00 2019-09-01 18:09:00 2019-09-01 15:00:00
...                                ...                 ...                 ...
2019-09-28 5  郭玉亮  2019-09-28 08:25:00 2019-09-28 17:42:00 2019-09-28 15:00:00
           7  刘明明  2019-09-28 08:55:00 2019-09-28 17:43:00 2019-09-28 15:00:00
           15 李颖   2019-09-28 08:56:00 2019-09-28 17:35:00 2019-09-28 15:00:00
           19 刘小虹  2019-09-28 08:38:00 2019-09-28 17:37:00 2019-09-28 15:00:00
           21 闫娟   2019-09-28 08:57:00 2019-09-28 17:32:00 2019-09-28 15:00:00

[151 rows x 3 columns]

In [69]:
df[]df[df['sign_in']>df['sign_in_tickle']]['sign_in']

day         ID  NAME
2019-09-05  23  李梦雪    2019-09-05 19:41:00
2019-09-08  2   王明浩    2019-09-08 17:44:00
            4   马雪一    2019-09-08 18:52:00
2019-09-09  4   马雪一    2019-09-09 17:44:00
2019-09-12  4   马雪一    2019-09-12 17:37:00
2019-09-18  4   马雪一    2019-09-18 17:32:00
2019-09-19  5   郭玉亮    2019-09-19 19:04:00
2019-09-21  5   郭玉亮    2019-09-21 17:59:00
2019-09-24  14  金德利    2019-09-24 17:13:00
2019-09-25  4   马雪一    2019-09-25 17:15:00
            14  金德利    2019-09-25 17:08:00
2019-09-27  5   郭玉亮    2019-09-27 18:05:00
            14  金德利    2019-09-27 17:19:00
2019-09-28  4   马雪一    2019-09-28 17:36:00
Name: sign_in, dtype: datetime64[ns]

In [97]:
#是否上班打卡
df['sign_to_in']=(df['sign_in']-df['sign_in_tickle']).dt.total_seconds()
df['sign_to_in']=df['sign_to_in'].apply(lambda x:1 if x<0 else 0)

In [98]:
#是否下班打卡
df['sign_to_out']=(df['sign_out']-df['sign_in_tickle']).dt.total_seconds()
df['sign_to_out']=df['sign_to_out'].apply(lambda x:1 if x>0 else 0)

In [99]:
df['sign_in']=df['sign_in'].astype('str')*df['sign_to_in']

In [100]:
df['sign_out']=df['sign_out'].astype('str')*df['sign_to_out']

In [113]:
df[df['sign_to_in']==0]

sign_in             sign_out      sign_in_tickle  \
day        ID NAME                                                    
2019-09-05 23 李梦雪           2019-09-05 19:41:00 2019-09-05 15:00:00   
2019-09-08 2  王明浩           2019-09-08 17:44:00 2019-09-08 15:00:00   
           4  马雪一           2019-09-08 18:52:00 2019-09-08 15:00:00   
2019-09-09 4  马雪一           2019-09-09 17:44:00 2019-09-09 15:00:00   
2019-09-12 4  马雪一           2019-09-12 17:37:00 2019-09-12 15:00:00   
2019-09-18 4  马雪一           2019-09-18 17:32:00 2019-09-18 15:00:00   
2019-09-19 5  郭玉亮           2019-09-19 19:04:00 2019-09-19 15:00:00   
2019-09-21 5  郭玉亮           2019-09-21 17:59:00 2019-09-21 15:00:00   
2019-09-24 14 金德利           2019-09-24 17:13:00 2019-09-24 15:00:00   
2019-09-25 4  马雪一           2019-09-25 17:15:00 2019-09-25 15:00:00   
           14 金德利           2019-09-25 17:08:00 2019-09-25 15:00:00   
2019-09-27 5  郭玉亮           2019-09-27 18:05:00 2019-09-27 15:00:00   
           14 金德利           2019-09-27 17:19:00 2019-09-27 15:00:00   
2019-09-28 4  马雪一           2019-09-28 17:36:00 2019-09-28 15:00:00   

                    sign_to_in  sign_to_out  
day        ID NAME                           
2019-09-05 23 李梦雪            0            1  
2019-09-08 2  王明浩            0            1  
           4  马雪一            0            1  
2019-09-09 4  马雪一            0            1  
2019-09-12 4  马雪一            0            1  
2019-09-18 4  马雪一            0            1  
2019-09-19 5  郭玉亮            0            1  
2019-09-21 5  郭玉亮            0            1  
2019-09-24 14 金德利            0            1  
2019-09-25 4  马雪一            0            1  
           14 金德利            0            1  
2019-09-27 5  郭玉亮            0            1  
           14 金德利            0            1  
2019-09-28 4  马雪一            0            1

In [275]:
df.to_csv('./上下班打卡记录.csv',encoding='utf_8_sig')

In [118]:
kaoqin1=pd.read_csv('./上下班打卡记录.csv',sep=',')

In [166]:
#设定标准上班时间
datestringlist=[x[:10]+' 09:00:00' for x in kaoqin1['sign_in_tickle'].astype('str')]
# tickle_in=[datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S') for x in datestringlist]
kaoqin1['sd_sign_in']=datestringlist

In [244]:
#计算真实上班时间与标准上班时间的差，以分钟计，为正表示没有迟到，为负则为迟到
# kaoqin1['sd_to_sign_in']=(kaoqin1['sd_sign_in'].astype('datetime64')-kaoqin1['sign_in'].astype('datetime64')).dt.total_seconds()/60
kaoqin1['sd_to_sign_in']=np.nan

In [238]:
#如果是星期六天返回true
weekday_tf=[x in (5,6) for x in kaoqin1['sign_in_tickle'].astype('datetime64').dt.weekday]

In [245]:
kaoqin1['sd_to_sign_in']=kaoqin1['sd_to_sign_in'].where(weekday_tf,
                              (kaoqin1['sd_sign_in'].astype('datetime64')-kaoqin1['sign_in'].astype('datetime64')).dt.total_seconds()/60)


In [205]:
#设定标准下班时间
datestringlist=[x[:10]+' 17:30:00' for x in kaoqin1['sign_in_tickle'].astype('str')]
# tickle_in=[datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S') for x in datestringlist]
kaoqin1['sd_sign_out']=datestringlist

In [246]:
#计算真实上班时间与标准下班时间的差，以分钟计
kaoqin1['sd_to_sign_out']=np.nan
kaoqin1['sd_to_sign_out']=kaoqin1['sd_to_sign_out'].where(weekday_tf,
                                                         (kaoqin1['sign_out'].astype('datetime64')-kaoqin1['sd_sign_out'].astype('datetime64')).dt.total_seconds()/60)


In [247]:

kaoqin1['是否迟到']=kaoqin1['sd_to_sign_in'].apply(lambda x : 1 if x<0 else 0)
kaoqin1['是否早退']=kaoqin1['sd_to_sign_out'].apply(lambda x : 1 if x<0 else 0)

In [259]:
#是否周末加班
# kaoqin1['是否周末加班']=1
kaoqin1['是否周末加班']=[1 if x==True else 0 for x in weekday_tf]

In [266]:
kaoqin1['是否工作日打卡']=1-kaoqin1['是否周末加班']

In [272]:
kaoqin1.to_csv('./考勤明细表是否上班迟到等.csv',encoding='utf_8_sig',index=False)

In [276]:
#实现月统计报表
#
kaoqin2=pd.read_csv('./考勤明细表是否上班迟到等.csv',sep=',')

In [281]:
kaoqin2['月份']=[x[:7] for x in kaoqin2['sign_in_tickle']]

In [283]:
kaoqin2

,day,ID,NAME,sign_in,sign_out,sign_in_tickle,sign_to_in,sign_to_out,sd_sign_in,sd_to_sign_in,sd_sign_out,sd_to_sign_out,是否迟到,是否早退,是否周末加班,是否工作日打卡,月份
0,2019-09-01,2,王明浩,2019-09-01 08:47:00,2019-09-01 17:31:00,2019-09-01 15:00:00,1,1,2019-09-01 09:00:00,NaN,2019-09-01 17:30:00,NaN,0,0,1,0,2019-09
1,2019-09-01,6,赵佳乐,2019-09-01 08:46:00,2019-09-01 20:10:00,2019-09-01 15:00:00,1,1,2019-09-01 09:00:00,NaN,2019-09-01 17:30:00,NaN,0,0,1,0,2019-09
2,2019-09-01,15,李颖,2019-09-01 08:29:00,2019-09-01 17:35:00,2019-09-01 15:00:00,1,1,2019-09-01 09:00:00,NaN,2019-09-01 17:30:00,NaN,0,0,1,0,2019-09
3,2019-09-01,16,孙超,2019-09-01 08:39:00,2019-09-01 17:31:00,2019-09-01 15:00:00,1,1,2019-09-01 09:00:00,NaN,2019-09-01 17:30:00,NaN,0,0,1,0,2019-09
4,2019-09-01,19,刘小虹,2019-09-01 08:33:00,2019-09-01 18:09:00,2019-09-01 15:00:00,1,1,2019-09-01 09:00:00,NaN,2019-09-01 17:30:00,NaN,0,0,1,0,2019-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,2019-09-28,5,郭玉亮,2019-09-28 08:25:00,2019-09-28 17:42:00,2019-09-28 15:00:00,1,1,2019-09-28 09:00:00,NaN,2019-09-28 17:30:00,NaN,0,0,1,0,2019-09
147,2019-09-28,7,刘明明,2019-09-28 08:55:00,2019-09-28 17:43:00,2019-09-28 15:00:00,1,1,2019-09-28 09:00:00,NaN,2019-09-28 17:30:00,NaN,0,0,1,0,2019-09
148,2019-09-28,15,李颖,2019-09-28 08:56:00,2019-09-28 17:35:00,2019-09-28 15:00:00,1,1,2019-09-28 09:00:00,NaN,2019-09-28 17:30:00,NaN,0,0,1,0,2019-09
149,2019-09-28,19,刘小虹,2019-09-28 08:38:00,2019-09-28 17:37:00,2019-09-28 15:00:00,1,1,2019-09-28 09:00:00,NaN,2019-09-28 17:30:00,NaN,0,0,1,0,2019-09


In [284]:
kaoqin2['每日迟到时长']=-kaoqin2['是否迟到']*kaoqin2['sd_to_sign_in']
kaoqin2['每日早退时长']=-kaoqin2['是否早退']*kaoqin2['sd_to_sign_out']


In [286]:
df_kaoqin2=kaoqin2.groupby(['ID','NAME','月份']).agg({
    '每日迟到时长':'sum',
    '每日早退时长':'sum',
    '是否迟到':'sum',
    '是否早退':'sum',
    '是否周末加班':'sum',
    '是否工作日打卡':'sum'
})


In [329]:
df_kaoqin2=df_kaoqin2.rename(columns={
    '每日迟到时长':'late_time',
    '每日早退时长':'leave_time',
    '是否迟到':'late_times',
    '是否早退':'leave_times',
    '是否周末加班':'rest_work_days',
    '是否工作日打卡':'day_work_days'
})

In [330]:
df_kaoqin2.to_csv('./考勤月统计报表.csv',encoding='utf_8_sig')

### 生成日历表
- 目的是计算出本月应该上多少天班

In [299]:
#计算2019-9月有多少工作日
#21天
work_day=pd.bdate_range('2019-09-01', '2019-09-30',freq='b').shape[0]

In [300]:
def day_deduct(late_time,work_day,sal):
    if (late_time>1 )&(late_time<120):
        return 30+sal/work_day/8/60*late_time
    elif (late_time>=120) &(late_time<240):
        return sal/work_day/2
    elif late_time>=240:
        return sal/work_day
    else:
        return 0
    

In [303]:
sals=pd.read_csv('./20201016课前资料/data/薪资表.csv',sep=',',encoding='gbk')

In [310]:
df_kaoqin2=df_kaoqin2.reset_index()

In [331]:
df_kaoqin3=df_kaoqin2.merge(sals[['EMPID','SAL']],left_on='ID',right_on='EMPID')

In [334]:
df_kaoqin3['day_deduct']=df_kaoqin3.apply(lambda x: day_deduct(x['late_time'],work_day,x['SAL']),axis=1)

In [335]:
df_kaoqin3['leave_deduct']=df_kaoqin3.apply(lambda x: day_deduct(x['leave_time'],work_day,x['SAL']),axis=1)

In [336]:
df_kaoqin3['week_fee']=df_kaoqin3['rest_work_days']*(df_kaoqin3['SAL']/work_day)*2

In [339]:
df_kaoqin3['real_fee']=np.round(df_kaoqin3['SAL']-df_kaoqin3['day_deduct']-df_kaoqin3['leave_deduct']+df_kaoqin3['week_fee'],2)


In [341]:
df_kaoqin3.to_csv('./实发工资.csv',encoding='utf_8_sig')